In [1]:
import tensorflow as tf
import os

# Get the absolute path to the current script's directory
current_dir = os.path.dirname(os.path.abspath(__file__))

# Construct the full path to the model file
model_path = os.path.join(current_dir, "updated_model.h5")

# Load the model using the full path
model = tf.keras.models.load_model(model_path)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'updated_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
import numpy as np

In [5]:
import cv2
import time
import numpy as np
import tensorflow as tf

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

start_time = time.time()
last_prediction_time = 0
prediction_interval = 4

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    current_time = time.time()

    for (x, y, w, h) in faces:
        if current_time - last_prediction_time > prediction_interval:
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (200, 200)).astype('float32') / 255.0
            face_resized = np.expand_dims(face_resized, axis=0)

            gender_pred, race_pred, age_pred = model.predict(face_resized)
            stored_gender = "Male" if gender_pred[0] < 0.5 else "Female"
            stored_age = f"{age_pred[0][0]:.2f}"
            last_prediction_time = current_time

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"Gender: {stored_gender}, Age: {stored_age}",
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Live Video Feed', frame)

    if time.time() - start_time > 10:
        print("Exiting after 10 seconds...")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting manually...")
        break

cap.release()
cv2.destroyAllWindows()

Error: Could not open webcam.
Failed to capture frame. Exiting...
